# CytoTable (convert) and Pycytominer (SingleCells.merge_single_cells) Performance Comparisons

This notebook explores CytoTable (convert) and Pycytominer (SingleCells.merge_single_cells) usage with datasets of varying size to help describe performance impacts.

In [1]:
import io
import itertools
import json
import os
import pathlib
import subprocess
import tokenize
from datetime import datetime

import pandas as pd
import plotly.express as px
import plotly.io as pio
from IPython.display import Image
from utilities import get_system_info

# set plotly default theme
pio.templates.default = "simple_white"

In [2]:
# show the system information
_ = get_system_info(show_output=True)


System Information:
Operating System: Linux
Machine Type: x86_64
Processor: x86_64
CPU Cores (Logical): 16
CPU Cores (Physical): 8
Total RAM (GB): 62.68
Python Version: 3.11.9


In [3]:
# observe the virtual env for dependency inheritance with memray
# from subprocedure calls
"/".join(
    subprocess.run(
        [
            "which",
            "memray",
        ],
        capture_output=True,
        check=True,
    )
    # decode bytestring as utf-8
    .stdout.decode("utf-8")
    # remove personal file structure
    .split("/")[6:]
    # replace final newline
).replace("\n", "")

'cytotable-benchmarks-2FApAEcz-py3.11/bin/memray'

In [4]:
# target file or table names
image_dir = "images"
examples_dir = "examples"
join_read_time_image = (
    f"{image_dir}/cytotable-and-pycytominer-comparisons-join-completion-time.png"
)
join_mem_size_image = (
    f"{image_dir}/cytotable-and-pycytominer-comparisons-join-memory-size.png"
)
example_files_list = [
    f"{examples_dir}/cytotable_convert_nf1_multiprocess.py",
    f"{examples_dir}/cytotable_convert_nf1_multithread.py",
    f"{examples_dir}/pycytominer_merge_nf1.py",
]
example_data_list = [
    f"{examples_dir}/data/all_cellprofiler.sqlite",
    f"{examples_dir}/data/all_cellprofiler-x2.sqlite",
    f"{examples_dir}/data/all_cellprofiler-x4.sqlite",
    f"{examples_dir}/data/all_cellprofiler-x8.sqlite",
    f"{examples_dir}/data/all_cellprofiler-x16.sqlite",
    f"{examples_dir}/data/all_cellprofiler-x32.sqlite",
    f"{examples_dir}/data/all_cellprofiler-x64.sqlite",
    f"{examples_dir}/data/all_cellprofiler-x128.sqlite",
    f"{examples_dir}/data/all_cellprofiler-x256.sqlite",
    f"{examples_dir}/data/all_cellprofiler-x512.sqlite",
]

# format for memray time strings
tformat = "%Y-%m-%d %H:%M:%S.%f%z"

In [5]:
# avoid a "cold start" for tested packages by using them before benchmarks
for example_file in example_files_list:
    run = subprocess.run(
        ["python", example_file, example_data_list[0]],
        check=True,
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL,
    )

In [6]:
# result list for storing data
results = []

# loop for iterating over examples and example data
# and gathering data about operations on them
for example_file, example_data in itertools.product(
    example_files_list,
    example_data_list,
):
    target_bin = f"{example_file}_with_{example_data.replace(f'{examples_dir}/data/','')}.memray.bin"
    target_json = f"{target_bin}.json"
    memray_run = subprocess.run(
        [
            "memray",
            "run",
            "--follow-fork",
            "--output",
            target_bin,
            "--force",
            example_file,
            example_data,
        ],
        # avoiding block buffering
        # which sometimes causes
        # hanging processes
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL,
        check=True,
    )

    memray_stats = subprocess.run(
        [
            "memray",
            "stats",
            "--json",
            "--output",
            target_json,
            "--force",
            target_bin,
        ],
        capture_output=True,
        check=True,
    )

    # open the json data
    with open(target_json) as memray_json_file:
        memray_data = json.load(memray_json_file)

    # append data to the result list
    results.append(
        {
            # general information about the dataframe
            "file_input": example_file.replace(f"{examples_dir}/", ""),
            "data_input": example_data,
            # information about pandas
            "time_duration (secs)": (
                datetime.strptime(memray_data["metadata"]["end_time"], tformat)
                - datetime.strptime(memray_data["metadata"]["start_time"], tformat)
            ).total_seconds(),
            "total_memory (bytes)": memray_data["total_bytes_allocated"],
        }
    )

    # cleanup
    pathlib.Path(target_bin).unlink(missing_ok=True)
    pathlib.Path(target_json).unlink(missing_ok=True)

df_results = pd.DataFrame(results)
df_results

,file_input,data_input,time_duration (secs),total_memory (bytes)
0,cytotable_convert_nf1_multiprocess.py,examples/data/all_cellprofiler.sqlite,15.407,487278299
1,cytotable_convert_nf1_multiprocess.py,examples/data/all_cellprofiler-x2.sqlite,15.558,490129249
2,cytotable_convert_nf1_multiprocess.py,examples/data/all_cellprofiler-x4.sqlite,16.075,490077017
3,cytotable_convert_nf1_multiprocess.py,examples/data/all_cellprofiler-x8.sqlite,16.052,494185511
4,cytotable_convert_nf1_multiprocess.py,examples/data/all_cellprofiler-x16.sqlite,16.234,494236242
5,cytotable_convert_nf1_multiprocess.py,examples/data/all_cellprofiler-x32.sqlite,16.868,492450897
6,cytotable_convert_nf1_multiprocess.py,examples/data/all_cellprofiler-x64.sqlite,18.957,492190816
7,cytotable_convert_nf1_multiprocess.py,examples/data/all_cellprofiler-x128.sqlite,21.875,491489501
8,cytotable_convert_nf1_multiprocess.py,examples/data/all_cellprofiler-x256.sqlite,28.404,491775844
9,cytotable_convert_nf1_multiprocess.py,examples/data/all_cellprofiler-x512.sqlite,41.226,480997699


In [7]:
# add columns for data understandability in plots


def get_file_size_mb(file_path):
    """
    Gather filesize given a file_path
    """
    try:
        return pathlib.Path(file_path).stat().st_size / 1024 / 1024
    except FileNotFoundError:
        return None


# memory usage in MB
df_results["total_memory (GB)"] = (
    df_results["total_memory (bytes)"] / 1024 / 1024 / 1024
)

# data input size additions for greater context
df_results["data_input_size_mb"] = df_results["data_input"].apply(get_file_size_mb)
df_results["data_input_with_size"] = (
    df_results["data_input"]
    + " ("
    + round(df_results["data_input_size_mb"]).astype("int64").astype("str")
    + " MB)"
)

# rename data input to simplify
df_results["data_input_renamed"] = (
    df_results["data_input_with_size"]
    .str.replace(f"{examples_dir}/data/", "")
    .str.replace("all_cellprofiler", "input")
)
df_results

,file_input,data_input,time_duration (secs),total_memory (bytes),total_memory (GB),data_input_size_mb,data_input_with_size,data_input_renamed
0,cytotable_convert_nf1_multiprocess.py,examples/data/all_cellprofiler.sqlite,15.407,487278299,0.453813,3.843750,examples/data/all_cellprofiler.sqlite (4 MB),input.sqlite (4 MB)
1,cytotable_convert_nf1_multiprocess.py,examples/data/all_cellprofiler-x2.sqlite,15.558,490129249,0.456468,7.578125,examples/data/all_cellprofiler-x2.sqlite (8 MB),input-x2.sqlite (8 MB)
2,cytotable_convert_nf1_multiprocess.py,examples/data/all_cellprofiler-x4.sqlite,16.075,490077017,0.456420,15.031250,examples/data/all_cellprofiler-x4.sqlite (15 MB),input-x4.sqlite (15 MB)
3,cytotable_convert_nf1_multiprocess.py,examples/data/all_cellprofiler-x8.sqlite,16.052,494185511,0.460246,29.937500,examples/data/all_cellprofiler-x8.sqlite (30 MB),input-x8.sqlite (30 MB)
4,cytotable_convert_nf1_multiprocess.py,examples/data/all_cellprofiler-x16.sqlite,16.234,494236242,0.460293,59.765625,examples/data/all_cellprofiler-x16.sqlite (60 MB),input-x16.sqlite (60 MB)
5,cytotable_convert_nf1_multiprocess.py,examples/data/all_cellprofiler-x32.sqlite,16.868,492450897,0.458631,119.390625,examples/data/all_cellprofiler-x32.sqlite (119...,input-x32.sqlite (119 MB)
6,cytotable_convert_nf1_multiprocess.py,examples/data/all_cellprofiler-x64.sqlite,18.957,492190816,0.458388,238.683594,examples/data/all_cellprofiler-x64.sqlite (239...,input-x64.sqlite (239 MB)
7,cytotable_convert_nf1_multiprocess.py,examples/data/all_cellprofiler-x128.sqlite,21.875,491489501,0.457735,477.300781,examples/data/all_cellprofiler-x128.sqlite (47...,input-x128.sqlite (477 MB)
8,cytotable_convert_nf1_multiprocess.py,examples/data/all_cellprofiler-x256.sqlite,28.404,491775844,0.458002,954.632812,examples/data/all_cellprofiler-x256.sqlite (95...,input-x256.sqlite (955 MB)
9,cytotable_convert_nf1_multiprocess.py,examples/data/all_cellprofiler-x512.sqlite,41.226,480997699,0.447964,1909.335938,examples/data/all_cellprofiler-x512.sqlite (19...,input-x512.sqlite (1909 MB)


In [8]:
# build cols for split reference in the plot
df_results["cytotable_time_duration (multiprocess) (secs)"] = df_results[
    df_results["file_input"] == "cytotable_convert_nf1_multiprocess.py"
]["time_duration (secs)"]
df_results["cytotable_total_memory (multiprocess) (GB)"] = df_results[
    df_results["file_input"] == "cytotable_convert_nf1_multiprocess.py"
]["total_memory (GB)"]
df_results["cytotable_time_duration (multithread) (secs)"] = df_results[
    df_results["file_input"] == "cytotable_convert_nf1_multithread.py"
]["time_duration (secs)"]
df_results["cytotable_total_memory (multithread) (GB)"] = df_results[
    df_results["file_input"] == "cytotable_convert_nf1_multithread.py"
]["total_memory (GB)"]
df_results["pycytominer_time_duration (secs)"] = df_results[
    df_results["file_input"] == "pycytominer_merge_nf1.py"
]["time_duration (secs)"]
df_results["pycytominer_total_memory (GB)"] = df_results[
    df_results["file_input"] == "pycytominer_merge_nf1.py"
]["total_memory (GB)"]
df_results = (
    df_results.apply(lambda x: pd.Series(x.dropna().values))
    .drop(["file_input", "time_duration (secs)", "total_memory (GB)"], axis=1)
    .dropna()
)
df_results

,data_input,total_memory (bytes),data_input_size_mb,data_input_with_size,data_input_renamed,cytotable_time_duration (multiprocess) (secs),cytotable_total_memory (multiprocess) (GB),cytotable_time_duration (multithread) (secs),cytotable_total_memory (multithread) (GB),pycytominer_time_duration (secs),pycytominer_total_memory (GB)
0,examples/data/all_cellprofiler.sqlite,487278299,3.843750,examples/data/all_cellprofiler.sqlite (4 MB),input.sqlite (4 MB),15.407,0.453813,10.761,46.702225,5.895,0.456070
1,examples/data/all_cellprofiler-x2.sqlite,490129249,7.578125,examples/data/all_cellprofiler-x2.sqlite (8 MB),input-x2.sqlite (8 MB),15.558,0.456468,10.646,46.638016,6.108,0.500124
2,examples/data/all_cellprofiler-x4.sqlite,490077017,15.031250,examples/data/all_cellprofiler-x4.sqlite (15 MB),input-x4.sqlite (15 MB),16.075,0.456420,10.582,46.741235,6.616,0.587669
3,examples/data/all_cellprofiler-x8.sqlite,494185511,29.937500,examples/data/all_cellprofiler-x8.sqlite (30 MB),input-x8.sqlite (30 MB),16.052,0.460246,10.863,46.971186,7.652,0.765980
4,examples/data/all_cellprofiler-x16.sqlite,494236242,59.765625,examples/data/all_cellprofiler-x16.sqlite (60 MB),input-x16.sqlite (60 MB),16.234,0.460293,11.450,47.176293,9.778,1.124454
5,examples/data/all_cellprofiler-x32.sqlite,492450897,119.390625,examples/data/all_cellprofiler-x32.sqlite (119...,input-x32.sqlite (119 MB),16.868,0.458631,11.931,47.399063,13.962,1.905748
6,examples/data/all_cellprofiler-x64.sqlite,492190816,238.683594,examples/data/all_cellprofiler-x64.sqlite (239...,input-x64.sqlite (239 MB),18.957,0.458388,13.794,48.152875,23.285,3.402082
7,examples/data/all_cellprofiler-x128.sqlite,491489501,477.300781,examples/data/all_cellprofiler-x128.sqlite (47...,input-x128.sqlite (477 MB),21.875,0.457735,17.177,49.726028,43.379,6.364014
8,examples/data/all_cellprofiler-x256.sqlite,491775844,954.632812,examples/data/all_cellprofiler-x256.sqlite (95...,input-x256.sqlite (955 MB),28.404,0.458002,23.420,54.151379,81.576,12.343289
9,examples/data/all_cellprofiler-x512.sqlite,480997699,1909.335938,examples/data/all_cellprofiler-x512.sqlite (19...,input-x512.sqlite (1909 MB),41.226,0.447964,42.073,69.055176,165.313,24.879804


In [9]:
# read time chart
fig = px.line(
    df_results,
    y=[
        "cytotable_time_duration (multiprocess) (secs)",
        "cytotable_time_duration (multithread) (secs)",
        "pycytominer_time_duration (secs)",
    ],
    x="data_input_renamed",
    title="CytoTable and Pycytominer SQLite Processing Time Comparison",
    labels={"data_input_renamed": "Input File", "value": "Seconds"},
    height=500,
    width=900,
    symbol_sequence=["diamond"],
    color_discrete_sequence=[
        px.colors.qualitative.Vivid[6],
        px.colors.qualitative.Vivid[7],
        px.colors.qualitative.Vivid[4],
    ],
)

# rename the lines for the legend
newnames = {
    "cytotable_time_duration (multiprocess) (secs)": "CytoTable (multiprocess)",
    "cytotable_time_duration (multithread) (secs)": "CytoTable (multithread)",
    "pycytominer_time_duration (secs)": "Pycytominer",
}
# referenced from: https://stackoverflow.com/a/64378982
fig.for_each_trace(
    lambda t: t.update(
        name=newnames[t.name],
        legendgroup=newnames[t.name],
        hovertemplate=t.hovertemplate.replace(t.name, newnames[t.name]),
    )
)

# update the legend
fig.update_layout(
    legend_title_text="",
    legend=dict(x=0.01, y=0.98, bgcolor="rgba(255,255,255,0.8)"),
    font=dict(
        size=16,  # global font size
    ),
)
# fig.update_xaxes(range=[-0.03, 5.2])
fig.update_traces(mode="lines+markers")

fig.write_image(join_read_time_image)
fig.write_image(join_read_time_image.replace(".png", ".svg"))
Image(url=join_read_time_image.replace(".png", ".svg"))

In [10]:
# memory size

fig = px.line(
    df_results,
    y=[
        "cytotable_total_memory (multiprocess) (GB)",
        "cytotable_total_memory (multithread) (GB)",
        "pycytominer_total_memory (GB)",
    ],
    x="data_input_renamed",
    title="CytoTable and Pycytominer SQLite Total Memory Consumption",
    labels={"data_input_renamed": "Input File", "value": "Gigabytes (GB)"},
    height=500,
    width=900,
    symbol_sequence=["diamond"],
    color_discrete_sequence=[
        px.colors.qualitative.Vivid[6],
        px.colors.qualitative.Vivid[7],
        px.colors.qualitative.Vivid[4],
    ],
)

# rename the lines for the legend
newnames = {
    "cytotable_total_memory (multiprocess) (GB)": "CytoTable (multiprocess)",
    "cytotable_total_memory (multithread) (GB)": "CytoTable (multithread)",
    "pycytominer_total_memory (GB)": "Pycytominer",
}
# referenced from: https://stackoverflow.com/a/64378982
fig.for_each_trace(
    lambda t: t.update(
        name=newnames[t.name],
        legendgroup=newnames[t.name],
        hovertemplate=t.hovertemplate.replace(t.name, newnames[t.name]),
    )
)

# update the legend
fig.update_layout(
    legend_title_text="",
    legend=dict(x=0.01, y=0.98, bgcolor="rgba(255,255,255,0.8)"),
    font=dict(
        size=16,  # global font size
    ),
)
fig.update_traces(mode="lines+markers")


fig.write_image(join_mem_size_image)
fig.write_image(join_mem_size_image.replace(".png", ".svg"))
Image(url=join_mem_size_image.replace(".png", ".svg"))